# Followers/followings lists of an Instagram account

In [ ]:
pip install selenium webdriver_manager

In [2]:
import pandas as pd
import re
from time import sleep
import os
import logging

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

In [3]:
# Constants
username = "..."
password = "..."

In [4]:
# Silent download of drivers
logging.getLogger('WDM').setLevel(logging.NOTSET)
os.environ['WDM_LOG'] = 'False'

# Driver set-up
service = Service()
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)

In [12]:
# Log in and make the profile page fully functional
def access_page(page_url):
    driver.get(page_url)
    sleep(2)
    try:
        driver.find_element(By.XPATH, "//a[contains(text(),'Log in')]").click()
        sleep(2)
        driver.find_element(By.XPATH, "//input[@aria-label='Phone number, username or email address']").send_keys(username) 
        sleep(1)
        driver.find_element(By.XPATH, "//input[@aria-label='Password']").send_keys(password)   
        sleep(1)
        driver.find_element(By.XPATH, '//button[@type="submit"]').click()
        sleep(5)
    except:
        return
    
    try:
        driver.find_element(By.XPATH, "//div[contains(text(),'Not now')]").click()
    except:
        return

In [13]:
# Scroll the inner pop-up window to get all followers/followings
def inner_scroll():
    inner_window = driver.find_element(By.XPATH, "//div[@class='_aano']")
    counter = len(driver.find_elements(By.XPATH, "//button"))
    while True:
        driver.execute_script(
            'arguments[0].scrollTop = arguments[0].scrollTop + document.querySelector("._aano").scrollHeight;',
            inner_window,
        )
        sleep(3)
        local_counter = len(driver.find_elements(By.XPATH, "//button"))
        # stop when the count of 'subscribe' buttons doesn't change
        if local_counter == counter:
            break
        counter = local_counter

In [14]:
""" 
Main function
- option is either 'followers' or 'following'
""" 
def get_follows(option, page_url): 
    access_page(page_url)
    sleep(5)
    #driver.find_element(By.XPATH, "//span[contains(text(),'followers')]").click()
    driver.find_element(By.PARTIAL_LINK_TEXT, option).click()
    sleep(3)
    
    # scrolling down the pop-up window
    inner_scroll()
    
    links = driver.find_elements(By.XPATH, "//a[@role='link']")
    # all links on page
    urls = [f.get_attribute('href') for f in links]  
    # only links to profiles
    follows_urls = list(filter(
    lambda x: (x.count('/') == 4) and 
        (not re.search(f"/explore/|/reels/|/inbox/|/followers/|/following/|/{username}/|/p/|/web/|/legal/|threads|facebook|/about/", x)), 
                      list(set(urls))
                          )
                     )
    return follows_urls

In [ ]:
profile = "https://www.instagram.com/.../"

In [ ]:
followers = get_follows("followers", profile)
followers

In [9]:
len(followers)

34

In [ ]:
followings = get_follows("following", profile)
followings

In [16]:
len(followings)

69